In [1]:
import os

from vitessce import (
    VitessceConfig,
    Component as cm,
    CoordinationType as ct,
    MultiImageWrapper,
)

In [2]:
import uuid
from tempfile import tempdir
import spatialdata

sdata = spatialdata.datasets.blobs()

sdata.write( os.path.join( tempdir, f"sdata_{uuid.uuid4()}" )  )

/Users/arnedf/miniconda3/envs/vitessce/lib/python3.12/site-packages/dask/dataframe/__init__.py:31: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  warnings.warn(
/Users/arnedf/miniconda3/envs/vitessce/lib/python3.12/site-packages/anndata/_core/anndata.py:401: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
  warnings.warn(
/Users/arnedf/miniconda3/envs/vitessce/lib/python3.12/site-packages/spatialdata/models/models.py:1053: UserWarning: Converting `region_key: region` to categorical dtype.
  return convert_region_column_to_categorical(adata)


INFO     The Zarr backing store has been changed from None the new file path:                                      
         /var/folders/q5/7yhs0l6d0x771g7qdbhvkvmr0000gp/T/sdata_c7ce6b92-1532-4e9d-a471-5dddd7be1dcb               


In [3]:
output_path_img = os.path.join( sdata.path, "images", "blobs_image" )

In [4]:
from vitessce import OmeZarrWrapper

vc = VitessceConfig(schema_version="1.0.15", name='liver', description='Resolve liver')

dataset = vc.add_dataset(name='Liver dataset').add_object(
    MultiImageWrapper(
        image_wrappers=[
            OmeZarrWrapper(img_path=output_path_img, is_bitmask=False, name='image'),
        ],
 )
)

In [5]:
spatial_plot = vc.add_view(cm.SPATIAL, dataset=dataset)
layer_controller = vc.add_view(cm.LAYER_CONTROLLER, dataset=dataset)

In [6]:
vc.link_views([spatial_plot, layer_controller], [ct.SPATIAL_ZOOM, ct.SPATIAL_TARGET_X, ct.SPATIAL_TARGET_Y])

vc.layout((spatial_plot | layer_controller) );

In [7]:
vw = vc.widget()
vw

VitessceWidget(config={'version': '1.0.15', 'name': 'liver', 'description': 'Resolve liver', 'datasets': [{'ui…

### upload to bucket, and get the config_dict

In [ ]:
import boto3

s3 = boto3.resource(
    service_name='s3',
    endpoint_url="https://objectstor.vib.be",
    aws_access_key_id=os.environ['ACCESS_KEY'],
    aws_secret_access_key=os.environ['SECRET_KEY'],
)

config_dict = vc.export(to='S3', s3=s3, bucket_name="spatial-hackathon-private", prefix='_test_vitessce_1')

In [23]:
import json

with open( "spatialdata_blobs.json", "w" ) as f: # this .json should be uploaded to the bucket
    json.dump(config_dict, f, indent=4)

In [25]:
# unfortunately url is not correct, so we need to change 'https://spatial-hackathon-private.s3.amazonaws.com' to 'https://objectstor.vib.be/spatial-hackathon-private'.

config_dict

{'version': '1.0.15',
 'name': 'liver',
 'description': 'Resolve liver',
 'datasets': [{'uid': 'A',
   'name': 'Liver dataset',
   'files': [{'fileType': 'raster.json',
     'options': {'schemaVersion': '0.0.2',
      'usePhysicalSizeScaling': False,
      'images': [{'name': 'image',
        'type': 'ome-zarr',
        'url': 'https://spatial-hackathon-private.s3.amazonaws.com/_test_vitessce_1/A/0/d14db50d-e887-4660-b7f5-fa3bc2fd98b7.ome.zarr'}],
      'renderLayers': ['image']}}]}],
 'coordinationSpace': {'dataset': {'A': 'A'},
  'spatialZoom': {'A': None},
  'spatialTargetX': {'A': None},
  'spatialTargetY': {'A': None}},
 'layout': [{'component': 'spatial',
   'coordinationScopes': {'dataset': 'A',
    'spatialZoom': 'A',
    'spatialTargetX': 'A',
    'spatialTargetY': 'A'},
   'x': 0,
   'y': 0,
   'w': 6,
   'h': 12},
  {'component': 'layerController',
   'coordinationScopes': {'dataset': 'A',
    'spatialZoom': 'A',
    'spatialTargetX': 'A',
    'spatialTargetY': 'A'},
   'x':